In [1]:
import pymysql
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from tqdm.autonotebook import tqdm
from pandarallel import pandarallel
import matplotlib.pyplot as plt

/home/ddu/.python-env3/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  """


In [2]:
from functional import seq
from functional.util import compose
from IPython.core.display import display, HTML
import lxml.html
import re
import string

In [3]:
DATA_DIR = "/home/ddu/projects/data/wiki_dump"

In [88]:
# OLD method
# def html_2_text0(html_str):
#     if pd.isnull(html_str):
#         return ""
#     try:
#         return " ".join(lxml.html.fromstring(html_str).itertext()).replace("\xa0", " ")
#     except Exception as e:
#         return None

In [5]:
public_doc = pd.read_csv(f"{DATA_DIR}/public_doc.csv")

In [8]:
public_doc.columns

Index(['document_id', 'searchin_id', 'title', 'name', 'privacy_level',
       'description', 'stripped_url', 'transcript', 'email', 'inherit_from_id',
       'inheritance_type', 'version', 'namespace', 'date_created',
       'date_updated', 'word_count'],
      dtype='object')

In [233]:
def remove_script(root):
    elements = root.xpath("//script")
    for e in elements:
        text = e.tail
        parent = e.getparent()
        if parent is not None:
            previous = r.getprevious()
            if previous is not None:
                previous.tail = text
            else:
                parent.text = "{}{}".format(parent.text or "", " {}".format(text) if text else "")
            parent.remove(e)

In [234]:
# inline_tags = ('a', 'abbr', 'acronym', 'b', 'bdi', 'bdo', 'big', 'cite', 'code', 'dfn', 'em', 'i', 'kbd',
#                    'label', 'mark', 'nav', 'output', 'progress', 'q', 's', 'slot', 'small', 'span', 'strong',
#                    'sub', 'sup', 'time', 'tt', 'var', 'wbr')
inline_tags = ('a', 'abbr', 'acronym', 'b', 'bdi', 'bdo', 'big', 'cite', 'dfn', 'em', 'i', 'kbd',
                   'label', 'mark', 'q', 's', 'slot', 'small', 'span', 'strong', "samp", "img"
                   'sub', 'sup', 'time', 'tt', 'var', 'wbr', 'u', 'del', 'ins')
def remove_inline_formating(root, tags=inline_tags):
    tag_regex = " or ".join([f"self::{x}" for x in tags])
    elements = root.xpath("//*[{}]".format(tag_regex))
    for e in elements[::-1]:
        if e.text and e.tail:
            text = f"{e.text}{e.tail}"
        else:
            text = e.text or e.tail
        parent = e.getparent()
        if parent is not None:
            previous = e.getprevious()
            if previous is not None:
                if previous.tail and text:
                    previous.tail = f"{previous.tail}{text}"
                else:
                    previous.tail = previous.tail or text
            else:
                if parent.text and text:
                    parent.text = f"{parent.text}{text}"
                else:
                    parent.text = parent.text or text
            parent.remove(e)

In [235]:
def html_2_textpieces(html_str):
    if pd.isnull(html_str):
        return ""
    try:
        r = lxml.html.fromstring(html_str)
        remove_script(r)
        remove_inline_formating(r)
        return [
            x.replace("\xa0", " ").strip()
            for x in r.itertext()
            if x.replace("\xa0", " ").strip() != ""
        ]
    except Exception as e:
        return None

In [247]:
test1 = "<div> It is a very good <strong>man</strong> with <i>U</i> very good arms</div>"
test2 = """
<div><h2> Title</h2><span style="color: red"><strong>Note</strong></span> that <u>something</u> could be <strong>very</strong> wrong.</div>
"""

In [248]:
display(HTML(test1))
print(html_2_textpieces(test1))

['It is a very good man with U very good arms']


In [249]:
display(HTML(test2))
print(html_2_textpieces(test2))

['Title', 'Note that something could be very wrong.']


In [250]:
i = 78
text = html_2_text(public_doc.iloc[i].description)
for x in text.split("\n"):
    print("--{}".format(x))

--Reporting latency issues:
--Please report it to us by using the following steps:
--1. Open up Chrome.
--2. On a Mac, press Command-Opt-i to open up the Developer Tools. On a PC, it's Control-Shift-i.
--3. Make sure you click on the Network tab. If you need to, enable it, otherwise just leave it blank.
--4. With that browser, type http://business.linkedin.com into the URL window and go. The Network tab should start filling up with data, displaying all the requests it had to make and the various timings it took.
--5. When the page fully renders, control-click (or right-click on a PC) anywhere within the Network area and select Save as HAR with Content.
--6. Take a quick snapshot of the Network area (on a Mac you can use Command-Shift-3, on a PC you'll have to press PrintScreen and then paste into Paint).
--7. Create a JIRA Bug under SITES and title it "latency - YOUR LOCATION - URL" and include the screenshot and the HAR file.
--This will help us collect more solid data on performance 

In [251]:
# i = 167
display(HTML(public_doc.iloc[i].description))
root = lxml.html.fromstring(public_doc.iloc[i].description)

Reporting latency issues:
"Please report it to us by using the following steps:1. Open up Chrome.2. On a Mac, press Command-Opt-i to open up the Developer Tools. On a PC, it's Control-Shift-i.3. Make sure you click on the Network tab. If you need to, enable it, otherwise just leave it blank.4. With that browser, type http://business.linkedin.com into the URL window and go. The Network tab should start filling up with data, displaying all the requests it had to make and the various timings it took.5. When the page fully renders, control-click (or right-click on a PC) anywhere within the Network area and select Save as HAR with Content.6. Take a quick snapshot of the Network area (on a Mac you can use Command-Shift-3, on a PC you'll have to press PrintScreen and then paste into Paint).7. Create a JIRA Bug under SITES and title it ""latency - YOUR LOCATION - URL"" and include the screenshot and the HAR file.This will help us collect more solid data on performance that will inform our next steps on improving latency."


## Process wiki

In [275]:
pandarallel.initialize(progress_bar=True, nb_workers=2)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [276]:
import string
string.printable

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [277]:
text_piece_splitter = "\x0b"

In [278]:
def process_fun(s):
    return text_piece_splitter.join(
        html_2_textpieces(s)
    )

In [280]:
from tqdm.autonotebook import tqdm as tqdm

In [281]:
text_pieces = []
for s in tqdm(public_doc.description.values):
    text_pieces.append(process_fun(s))

In [282]:
public_doc.columns

Index(['document_id', 'searchin_id', 'title', 'name', 'privacy_level',
       'description', 'stripped_url', 'transcript', 'email', 'inherit_from_id',
       'inheritance_type', 'version', 'namespace', 'date_created',
       'date_updated', 'word_count'],
      dtype='object')

In [284]:
wiki_proc = public_doc[["document_id", "title", "name", "stripped_url", "namespace", "date_created", "date_updated"]].reset_index(drop=True)

In [285]:
len(text_pieces)

253697

In [286]:
wiki_proc.shape

(253697, 7)

In [287]:
wiki_proc["body_processed"] = text_pieces

In [288]:
wiki_proc.to_csv("./public_doc_processed.csv", index=False)